In [1]:
import requests
import json

In [2]:
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
# Generic Function to send GET requests to DraCor API

#corpusname:str -> []
def get(**kwargs):
    #corpusname=corpusname
    #playname=playname
    #apibase="https://dracor.org/api/"
    #method=method
    #parse_json: True
    
    #could set different apibase, e.g. https://staging.dracor.org/api/ [not recommended, pls use the production server]
    if "apibase" in kwargs:
        if kwargs["apibase"].endswith("/"):
            apibase = kwargs["apibase"]
        else:
            apibase = kwargs["apibase"] + "/"
    else:
        #use local API per default
        apibase = "http://dracor.org/api/"
    if "corpusname" in kwargs and "playname" in kwargs:
        # used for /api/corpora/{corpusname}/play/{playname}/
        if "method" in kwargs:
            request_url = apibase + "corpora/" + kwargs["corpusname"] + "/play/" + kwargs["playname"] + "/" + kwargs["method"]
        else:
            request_url = apibase + "corpora/" + kwargs["corpusname"] + "/play/" + kwargs["playname"]
    elif "corpusname" in kwargs and not "playname" in kwargs:
        if "method" in kwargs:
            request_url = apibase + "corpora/" + kwargs["corpusname"] + "/" + kwargs["method"]
        else:
            request_url = apibase + "corpora/" + kwargs["corpusname"] 
    elif "method" in kwargs and not "corpusname" in kwargs and not "playname" in kwargs:
            request_url = apibase + kwargs["method"]
            
    else: 
        #nothing set
        request = request_url = apibase + "info"
    
    #send the response
    r = requests.get(request_url)
    if r.status_code == 200:
        #success!
        if "parse_json" in kwargs:
            if kwargs["parse_json"] == True:
                json_data = json.loads(r.text)
                return json_data
            else:
                return r.text
        else:
            return r.text
    else:
        raise Exception("Request was not successful. Server returned status code: "  + str(r.status_code))

In [4]:
from dracor_api_response_schemas import is_valid

## /info

In [5]:
from dracor_api_response_schemas import InfoSchema

In [6]:
info_response = get(method="info",parse_json=True)

In [7]:
info_response

{'name': 'DraCor API',
 'version': '0.89.0',
 'status': 'beta',
 'existdb': '6.0.1'}

In [8]:
# Validate against the schema
# In case of an validation Error (returned value is False), set the flag "catch_error=False", which will return the 
# message from the exception
is_valid(info_response, InfoSchema)

INFO:root:Validation passed.


True

## /corpora

In [9]:
from dracor_api_response_schemas import CorpusInCorporaSchema

In [10]:
corpora_response_no_metrics = get(method="corpora", parse_json=True)

In [11]:
print("Validation of /corpora without metrics included:")
for corpus in corpora_response_no_metrics:
    if is_valid(corpus, CorpusInCorporaSchema):
        print("Data of corpus " + corpus["title"] + " passed validation.")
    else:
        print ("ERROR " + corpus["title"])

INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.


Validation of /corpora without metrics included:
Data of corpus Alsatian Drama Corpus passed validation.
Data of corpus Bashkir Drama Corpus passed validation.
Data of corpus Calderón Drama Corpus passed validation.
Data of corpus French Drama Corpus passed validation.
Data of corpus German Drama Corpus passed validation.
Data of corpus German Shakespeare Drama Corpus passed validation.
Data of corpus Greek Drama Corpus passed validation.
Data of corpus Hungarian Drama Corpus passed validation.
Data of corpus Italian Drama Corpus passed validation.
Data of corpus Roman Drama Corpus passed validation.
Data of corpus Russian Drama Corpus passed validation.
Data of corpus Shakespeare Drama Corpus passed validation.
Data of corpus Spanish Drama Corpus passed validation.
Data of corpus Swedish Drama Corpus passed validation.
Data of corpus Tatar Drama Corpus passed validation.


In [12]:
corpora_response_incl_metrics = get(method="corpora?include=metrics", parse_json=True)

In [13]:
print("Validation of /corpora with metrics included:")
for corpus in corpora_response_incl_metrics:
    if is_valid(corpus, CorpusInCorporaSchema):
        print("Data of corpus " + corpus["title"] + " passed validation.")
    else:
        print ("ERROR " + corpus["title"])

INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.


Validation of /corpora with metrics included:
Data of corpus Alsatian Drama Corpus passed validation.
Data of corpus Bashkir Drama Corpus passed validation.
Data of corpus Calderón Drama Corpus passed validation.
Data of corpus French Drama Corpus passed validation.
Data of corpus German Drama Corpus passed validation.
Data of corpus German Shakespeare Drama Corpus passed validation.
Data of corpus Greek Drama Corpus passed validation.
Data of corpus Hungarian Drama Corpus passed validation.
Data of corpus Italian Drama Corpus passed validation.
Data of corpus Roman Drama Corpus passed validation.
Data of corpus Russian Drama Corpus passed validation.
Data of corpus Shakespeare Drama Corpus passed validation.
Data of corpus Spanish Drama Corpus passed validation.
Data of corpus Swedish Drama Corpus passed validation.
Data of corpus Tatar Drama Corpus passed validation.


## /corpora/{corpusname} 

In [14]:
from dracor_api_response_schemas import CorpusSchema

In [15]:
corpus_response = get(corpusname="ger", parse_json=True)

In [16]:
#corpus_response

In [17]:
# is_valid(corpus_response, CorpusSchema, catch_error=False)
is_valid(corpus_response, CorpusSchema)

INFO:root:Validation passed.


True

In [18]:
#Validate all
corpusnames = []
for item in corpora_response_no_metrics:
    corpusnames.append(item["name"])

In [19]:
corpusnames

['als',
 'bash',
 'cal',
 'fre',
 'ger',
 'gersh',
 'greek',
 'hun',
 'ita',
 'rom',
 'rus',
 'shake',
 'span',
 'swe',
 'tat']

for corpusname in corpusnames:
    corpus_response = get(corpusname=corpusname, parse_json=True)
    print(f"Validating {corpusname}.")
    is_valid(corpus_response, CorpusSchema, catch_error=False)

## /corpora/{corpusname}/metadata

In [20]:
from dracor_api_response_schemas import PlayMetadataSchema

In [21]:
# metadata_plays_of_corpus_response = get(method="metadata",corpusname="ger",parse_json=True)

In [22]:
# metadata_plays_of_corpus_response[0]

In [23]:
#is_valid(metadata_plays_of_corpus_response[0], PlayMetadataSchema, catch_error=False)

for play in metadata_plays_of_corpus_response:
    is_valid(play, PlayMetadataSchema, catch_error=True)  

# Validate all
for corpusname in corpusnames:
    print(f"Validating corpus {corpusname}.")
    plays_metadata = get(method="metadata",corpusname=corpusname,parse_json=True)
    
    for play in plays_metadata:
        print(f"Validating {play['id']}.")
        is_valid(play, PlayMetadataSchema, catch_error=False)  
    

OK, all valid... to test, change the cell above back from markdown to code

## /corpora/{corpusname}/play/{playname}

In [24]:
from dracor_api_response_schemas import PlaySchema

In [25]:
def validate_play(corpusname: str = None, playname:str = None, catch_error = True):
    data = get(corpusname=corpusname, playname=playname,parse_json=True)
    return is_valid(data, PlaySchema, catch_error=catch_error) 
    

In [26]:
#validate_play(corpusname="ger", playname="lessing-emilia-galotti")

In [27]:
corpusnames_no_cal = corpusnames
corpusnames_no_cal.remove("cal")

for corpusname in corpusnames_no_cal:
    #print(corpusname)
    plays = get(corpusname=corpusname,parse_json=True)["dramas"]
    playnames = []
    for play in plays:
        playnames.append(play["name"])
    #print(playnames)
    
    for playname in playnames:
        print(f"Validating {corpusname} – {playname}.")
        validate_play(corpusname=corpusname, playname=playname, catch_error=False)

# do it separately..
# UNKWON ... stop at first stupid error
for corpusname in ["hun"]:
    #print(corpusname)
    plays = get(corpusname=corpusname,parse_json=True)["dramas"]
    playnames = []
    for play in plays:
        playnames.append(play["name"])
    #print(playnames)
    
    for playname in playnames:
        print(f"Validating {corpusname} – {playname}.")
        validate_play(corpusname=corpusname, playname=playname, catch_error=False)

for corpusname in ["ita"]:
    #print(corpusname)
    plays = get(corpusname=corpusname,parse_json=True)["dramas"]
    playnames = []
    for play in plays:
        playnames.append(play["name"])
    #print(playnames)
    
    for playname in playnames:
        print(f"Validating {corpusname} – {playname}.")
        validate_play(corpusname=corpusname, playname=playname, catch_error=False)

for corpusname in ["rom"]:
    #print(corpusname)
    plays = get(corpusname=corpusname,parse_json=True)["dramas"]
    playnames = []
    for play in plays:
        playnames.append(play["name"])
    #print(playnames)
    
    for playname in playnames:
        print(f"Validating {corpusname} – {playname}.")
        validate_play(corpusname=corpusname, playname=playname, catch_error=False)

for corpusname in ["rus"]:
    #print(corpusname)
    plays = get(corpusname=corpusname,parse_json=True)["dramas"]
    playnames = []
    for play in plays:
        playnames.append(play["name"])
    #print(playnames)
    
    for playname in playnames:
        print(f"Validating {corpusname} – {playname}.")
        validate_play(corpusname=corpusname, playname=playname, catch_error=False)

"""    {
      "id": "Demetrius_MND",
      "name": "Demetrius",
      "isGroup": false,
      "sex": "MAE"
    },

"""
for corpusname in ["shake"]:
    #print(corpusname)
    plays = get(corpusname=corpusname,parse_json=True)["dramas"]
    playnames = []
    for play in plays:
        playnames.append(play["name"])
    #print(playnames)
    
    for playname in playnames:
        print(f"Validating {corpusname} – {playname}.")
        validate_play(corpusname=corpusname, playname=playname, catch_error=False)

for corpusname in ["span"]:
    #print(corpusname)
    plays = get(corpusname=corpusname,parse_json=True)["dramas"]
    playnames = []
    for play in plays:
        playnames.append(play["name"])
    #print(playnames)
    
    for playname in playnames:
        print(f"Validating {corpusname} – {playname}.")
        validate_play(corpusname=corpusname, playname=playname, catch_error=False)

for corpusname in ["swe","tat"]:
    #print(corpusname)
    plays = get(corpusname=corpusname,parse_json=True)["dramas"]
    playnames = []
    for play in plays:
        playnames.append(play["name"])
    #print(playnames)
    
    for playname in playnames:
        print(f"Validating {corpusname} – {playname}.")
        validate_play(corpusname=corpusname, playname=playname, catch_error=False)

for corpusname in ["tat"]:
    #print(corpusname)
    plays = get(corpusname=corpusname,parse_json=True)["dramas"]
    playnames = []
    for play in plays:
        playnames.append(play["name"])
    #print(playnames)
    
    for playname in playnames:
        print(f"Validating {corpusname} – {playname}.")
        validate_play(corpusname=corpusname, playname=playname, catch_error=False)

## /corpora/{corpusname}/play/{playname}/metrics

In [28]:
from dracor_api_response_schemas import PlayMetricsSchema

In [29]:
def validate_play_metrics(corpusname: str = None, playname:str = None, catch_error = True):
    data = get(corpusname=corpusname, method="metrics", playname=playname,parse_json=True)
    return is_valid(data, PlayMetricsSchema, catch_error=catch_error) 

In [30]:
validate_play_metrics(corpusname="ger", playname="lessing-emilia-galotti")

INFO:root:Validation passed.


True

for corpusname in ["ger"]:
    #print(corpusname)
    plays = get(corpusname=corpusname,parse_json=True)["dramas"]
    playnames = []
    for play in plays:
        playnames.append(play["name"])
    #print(playnames)
    
    for playname in playnames:
        print(f"Validating {corpusname} – {playname}.")
        validate_play_metrics(corpusname=corpusname, playname=playname, catch_error=False)

In [31]:
for corpusname in corpusnames_no_cal:
    #print(corpusname)
    plays = get(corpusname=corpusname,parse_json=True)["dramas"]
    playnames = []
    for play in plays:
        playnames.append(play["name"])
    #print(playnames)
    
    for playname in playnames:
        print(f"Validating {corpusname} – {playname}.")
        validate_play_metrics(corpusname=corpusname, playname=playname, catch_error=False)

Validating als – arnold-der-pfingstmontag.
Validating als – bastian-e-sportschochzitt.
Validating als – bastian-dr-maischter-hett-gewunne.
Validating als – bastian-dr-hofnarr-heidideldum.
Validating als – bastian-dr-hans-im-schnokeloch.
Validating als – clemens-gift.
Validating als – clemens-chrischtowe.
Validating als – clemens-charlot.
Validating als – clemens-a-latzi-visit.
Validating als – fuchs-heimlichi-lieb.
Validating als – greber-sainte-cecile.
Validating als – greber-lucie.
Validating als – greber-d-jumpfer-prinzesse.
Validating als – hahn-jungi-madamme.
Validating als – hart-dr-poetisch-oscar.
Validating als – horsch-d-madam-fahrt-velo.
Validating als – jost-so-e-liederlicher-frack.
Validating als – jost-daa-im-narrehuss.
Validating als – kettner-uff-dr-hochzittsreis.
Validating als – kleisecker-dr-schnider-von-gambse.
Validating als – kuehne-bureaukrate.
Validating als – riff-sainte-barbe.
Validating als – riff-s-paradies.
Validating als – schneegans-steckelburjer-universit

Validating fre – boursault-esope.
Validating fre – boursault-fete-de-la-seine.
Validating fre – boursault-germanicus.
Validating fre – boursault-jaloux-endormi.
Validating fre – boursault-marie-stuard.
Validating fre – boursault-medecin-volant.
Validating fre – boursault-mort-vivant.
Validating fre – boursault-mots-a-la-mode.
Validating fre – boursault-portrait-du-peintre.
Validating fre – boursault-satire-des-satires.
Validating fre – boyera-caton.
Validating fre – boyer-agamemnon.
Validating fre – boyer-amours-jupiter-semele.
Validating fre – boyer-aristodeme.
Validating fre – boyer-artaxerce.
Validating fre – boyer-clotilde.
Validating fre – boyer-federic.
Validating fre – boyer-jeune-marius.
Validating fre – boyer-judith.
Validating fre – boyer-lisimene.
Validating fre – boyer-mort-enfants-brute.
Validating fre – boyer-oropaste.
Validating fre – boyer-porcie-romaine.
Validating fre – boyer-porus.
Validating fre – boyer-tiridate.
Validating fre – brohan-compter-sans-son-hote.
Valida

Validating fre – corneillep-rodogune.
Validating fre – corneillep-sertorius.
Validating fre – corneillep-sophonisbe.
Validating fre – corneillep-suite-menteur.
Validating fre – corneillep-suivante.
Validating fre – corneillep-surena.
Validating fre – corneillep-theodore.
Validating fre – corneillep-tite.
Validating fre – corneillep-toison.
Validating fre – corneillep-veuve-34.
Validating fre – corneillep-veuve.
Validating fre – dupaty-luce-salverte-bievre.
Validating fre – corneillet-amour-a-la-mode.
Validating fre – corneillet-antiochus.
Validating fre – corneillet-ariane.
Validating fre – corneillet-baron-d-albikrac.
Validating fre – corneillet-berenice.
Validating fre – corneillet-berger-extravagant.
Validating fre – corneillet-bradamante.
Validating fre – corneillet-camma.
Validating fre – corneillet-charme-de-la-voix.
Validating fre – corneillet-circe.
Validating fre – corneillet-comte-d-essex.
Validating fre – corneillet-comtesse-orgueil.
Validating fre – corneillet-darius.
Valid

Validating fre – donneau-de-vise-vengeance-des-marquis.
Validating fre – donneau-de-vise-veuve-a-la-mode.
Validating fre – donneau-de-vise-zelinde.
Validating fre – dorat-celibataire.
Validating fre – dorat-regulus.
Validating fre – dorimond-amours-de-trapolin.
Validating fre – dorimond-comedie-de-la-comedie.
Validating fre – dorimond-ecole-des-cocus.
Validating fre – dorimond-femme-industrieuse.
Validating fre – dorneval-fuzelier-lesage-arlequin-roi-des-ogres.
Validating fre – dorneval-jugement-de-paris.
Validating fre – dorneval-lesage-amours-de-nanterre.
Validating fre – dorneval-lesage-couplets-en-proces.
Validating fre – dorneval-lesage-opera-comique.
Validating fre – dorneval-lesage-reine-de-barostan.
Validating fre – lesage-dorneval-ile-gougou.
Validating fre – archambault-comedie-a-l-impromptu.
Validating fre – archambault-on-fait-ce-qu-on-peut.
Validating fre – dorvo-lamentin.
Validating fre – dubosc-balance-d-etat.
Validating fre – du-ryer-alcimedon.
Validating fre – du-ryer-

Validating fre – gonnet-angelique.
Validating fre – gonnet-clementine.
Validating fre – gonnet-eugenie.
Validating fre – gonnet-euphemie.
Validating fre – gonnet-juliette-ou-la-danseuse.
Validating fre – gonnet-lisette.
Validating fre – gosse-auteur-dans-son-menage.
Validating fre – gouges-cherubin.
Validating fre – gouges-exclavage-des-noirs.
Validating fre – gouges-moliere-ninon.
Validating fre – goulard-agis.
Validating fre – goullinet-trois-aveugles.
Validating fre – gourmont-theodat.
Validating fre – graffigny-phaza.
Validating fre – graffigny-ziman-zenise.
Validating fre – grandval-eunuque.
Validating fre – grandval-luxurieux.
Validating fre – grandval-temperament.
Validating fre – gresset-edouard-iii.
Validating fre – labaume-messe-de-gnide.
Validating fre – grosclaude-peche-a-la-ligne.
Validating fre – grevin-cesar.
Validating fre – grevin-tresoriere.
Validating fre – gudin-lothaire.
Validating fre – guerin-de-bouscal-dom-quixote-2.
Validating fre – guerin-de-bouscal-dom-quixot

...

Validating ger – moritz-blunt-oder-der-gast.
Validating ger – mosenthal-die-lustigen-weiber-von-windsor.
Validating ger – mosenthal-das-goldene-kreuz.
Validating ger – mosenthal-die-koenigin-von-saba.
Validating ger – moser-krieg-oder-frieden.
Validating ger – moser-das-stiftungsfest.
Validating ger – musil-die-schwaermer.
Validating ger – mylius-die-schaeferinsel.
Validating ger – moeller-der-graf-von-walltron-oder-die-subordination.
Validating ger – moeller-sophie-oder-der-gerechte-fuerst.
Validating ger – moeser-die-tugend-auf-der-schaubuehne.
Validating ger – muehsam-judas.
Validating ger – muehsam-staatsraeson.
Validating ger – muellner-koenig-yngurd.
Validating ger – muellner-der-neun-und-zwanzigste-februar.
Validating ger – muellner-die-schuld.
Validating ger – nestroy-zu-ebener-erde-und-erster-stock.
Validating ger – nestroy-eulenspiegel.
Validating ger – nestroy-der-unbedeutende.
Validating ger – nestroy-das-haus-der-temperamente.
Validating ger – nestroy-judith-und-holofernes

Validating ger – vanoni-der-friseur.
Validating ger – vischer-faust.
Validating ger – voss-faust.
Validating ger – wagner-die-reue-nach-der-that.
Validating ger – wagner-die-kindermoerderin.
Validating ger – wagner-voltaire-am-abend-seiner-apotheose.
Validating ger – wagner-tannhaeuser.
Validating ger – wagner-parsifal.
Validating ger – wagner-goetterdaemmerung.
Validating ger – wagner-das-rheingold.
Validating ger – wagner-tristan-und-isolde.
Validating ger – wagner-siegfried.
Validating ger – wagner-lohengrin.
Validating ger – wagner-die-walkuere.
Validating ger – wagner-die-meistersinger-von-nuernberg.
Validating ger – wagner-der-fliegende-hollaender.
Validating ger – wedekind-tod-und-teufel.
Validating ger – wedekind-musik.
Validating ger – wedekind-koenig-nicolo.
Validating ger – wedekind-hidalla.
Validating ger – wedekind-fruehlings-erwachen.
Validating ger – wedekind-fritz-schwigerling.
Validating ger – wedekind-franziska.
Validating ger – wedekind-erdgeist.
Validating ger – wed

Validating ita – alamanni-tragedia-di-antigone.
Validating ita – alfieri-agamennone.
Validating ita – alfieri-antigone.
Validating ita – alfieri-bruto-primo.
Validating ita – alfieri-bruto-secondo.
Validating ita – alfieri-don-garzia.
Validating ita – alfieri-filippo.
Validating ita – alfieri-la-congiura-de-pazzi.
Validating ita – alfieri-maria-stuarda.
Validating ita – alfieri-merope.
Validating ita – alfieri-mirra.
Validating ita – alfieri-oreste.
Validating ita – alfieri-ottavia.
Validating ita – alfieri-polinice.
Validating ita – alfieri-rosmunda.
Validating ita – alfieri-saul.
Validating ita – alfieri-sofonisba.
Validating ita – alfieri-timoleone.
Validating ita – alfieri-virginia.
Validating ita – andreini-lo-schiavetto.
Validating ita – ariosto-cassaria.
Validating ita – ariosto-i-studenti.
Validating ita – ariosto-i-suppositi-red-in-versi.
Validating ita – ariosto-i-suppositi.
Validating ita – ariosto-il-negromante.
Validating ita – ariosto-la-cassaria-red-in-versi.
Validating 

Validating rus – kapnist-yabeda.
Validating rus – kedrin-rembrandt.
Validating rus – knyazhnin-chudaki.
Validating rus – knyazhnin-didona.
Validating rus – knyazhnin-hvastun.
Validating rus – knyazhnin-neudachnyj-primiritel.
Validating rus – knyazhnin-olga.
Validating rus – knyazhnin-rosslav.
Validating rus – knyazhnin-traur.
Validating rus – knyazhnin-vadim-novgorodskij.
Validating rus – kokoshkin-vospitanie.
Validating rus – kropotov-fomushka.
Validating rus – krylov-podschipa-ili-trumf.
Validating rus – krylov-amerikantsy.
Validating rus – krylov-filomela.
Validating rus – krylov-modnaja-lavka.
Validating rus – krylov-pirog.
Validating rus – krylov-prokazniki.
Validating rus – krylov-sonnyj-poroshok.
Validating rus – krylov-urok-dochkam.
Validating rus – lazhechnikov-okopirovalsya.
Validating rus – lensky-horosha-i-durna.
Validating rus – lermontov-maskarad.
Validating rus – lermontov-menschen-und-leidenschaften.
Validating rus – lermontov-strannyj-chelovek.
Validating rus – leskov-

Validating shake – the-taming-of-the-shrew.
Validating shake – the-tempest.
Validating shake – the-winter-s-tale.
Validating shake – timon-of-athens.
Validating shake – titus-andronicus.
Validating shake – troilus-and-cressida.
Validating shake – twelfth-night.
Validating shake – two-gentlemen-of-verona.
Validating span – clarin-teresa.
Validating span – dicenta-juan-jose.
Validating span – echegaray-arrastrarse.
Validating span – echegaray-mancha.
Validating span – galdos-casandra.
Validating span – galdos-electra.
Validating span – galdos-perfecta.
Validating span – lorca-bernarda.
Validating span – lorca-bodas.
Validating span – lorca-rosita.
Validating span – lorca-yerma.
Validating span – lorca-zapatera.
Validating span – munoz-conferencia.
Validating span – munoz-ortiz.
Validating span – munoz-pergaminos.
Validating span – munoz-refugio.
Validating span – unamuno-esfinge.
Validating span – unamuno-fedra.
Validating span – valera-asclepigenia.
Validating span – valera-atahualpa.
V